### Scrapper

In [166]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from collections import Counter
import csv
import pandas as pd

In [167]:
# Function to set up the Chrome WebDriver
def setup_webdriver():
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # Run Chrome in headless mode (optional)
    options.add_argument("--no-sandbox")
    return webdriver.Chrome(options=options)


In [168]:
# Function to scroll and load more job postings
def scroll_to_load_jobs(driver, num_pages):
    print("Scrolling Loading Function")
    print("---------------------------")
    for _ in range(num_pages):
        print(f"Scrolling to load more jobs - Page { _ + 1}")
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
        time.sleep(2)  # Wait for content to load


In [169]:
# Function to expand job description by clicking "show more" if available
def expand_description(driver):
    try:
        show_more_button = WebDriverWait(card, 10)\
            .until(EC.presence_of_element_located((By.CSS_SELECTOR, '.show-more-less-html__button')))

        show_more_button.click()
        time.sleep(2)  # Wait for the description to expand
    except Exception as e:
        pass  # No "show more" button found or error occurred


In [170]:
# Function to extract job description using JavaScript
def extract_job_description(driver):
    try:
        # Wait for the job description element to be present (you can adjust the timeout)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '.description')))

        # Execute JavaScript code to extract job description
        job_description = driver.execute_script("return document.querySelector('.description').textContent")
        
        return job_description
    except TimeoutException:
        return "Job description not found or couldn't be loaded"

In [171]:
# Function to extract skills from a job title
def extract_skills(description):
    # This is a basic example, you can extend this to match more skills
    skills = ['PostgreSQL','Snowflake','Databricks','Redshift','BigQuery','MongoDB','MySQL',
              'Kafka','Kinesis','PubSub','Pub/sub','event hub','Airflow','dbt','NiFi', 'Fivetran',
            'Collibra','Denodo','presto','Starburst','Immuta','PowerBI','Tableau','Looker',
             'Matillion','Alteryx','Informatica','Talend','EMR','Dataproc','Synapse']
    description = description.lower()
    skills_list = [skill for skill in skills if skill.lower() in description]
    print(skills_list)
    print("---------------------------")
    print("---------------------------")
    return skills_list


In [172]:
# Function to extract job details 
# https://yoksel.github.io/html-tree/en/
def extract_job_details(card):
    try:
        job_title_element = WebDriverWait(card, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.base-search-card__title'))
        )
        company_element = WebDriverWait(card, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.base-search-card__subtitle'))
        )
        description = WebDriverWait(card, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.base-card__full-link'))
        )
        location_element = WebDriverWait(card, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.job-search-card__location'))
        )
        time_element = WebDriverWait(card, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.job-search-card__listdate'))
        )
        
        
        company_name = company_element.text
        job_title = job_title_element.text
        job_link = description.get_attribute('href')
        location = location_element.text
        print(f"location:{location}")
        time =time_element.text
        print(f"Time:{time}")
        

        return job_title, company_name, job_link, location, time
    except Exception as e:
        print("Job details not found for this card.")
        return None, None, None, None, None

In [173]:
# Main function to scrape LinkedIn jobs
def scrape_linkedin_jobs(keyword, num_pages):
    job_skills = []
    
    post_list = []
    company_list = []
    link_list = []
    post_date = []
    job_location = []
    
    driver = setup_webdriver()
    
    url = f'https://www.linkedin.com/jobs/search/?keywords={keyword}'
    driver.get(url)

      
    scroll_to_load_jobs(driver, num_pages)

    print("After parsing url within Scrape function")
    print("---------------------------")
    
    job_cards = driver.find_elements(By.CSS_SELECTOR, '.base-card')
            

    for card in job_cards:
        job_title, company_name, job_link, location, time = extract_job_details(card)

        #appending all details to respective list
        post_list.append(job_title)
        company_list.append(company_name)
        link_list.append(job_link)
        job_location.append(location)
        post_date.append(time)
        
        
        if job_title is None:
            continue

        print(f"{job_title} at {company_name}")
        print(f"Job Link: {job_link}")
        
        job_driver = setup_webdriver()
        job_driver.get(job_link)

        expand_description(job_driver)  # for clicking show more
        job_description = extract_job_description(job_driver) # extracting job description 
        extracted_skills = extract_skills(job_description) # extracting list of skills that match our list
        
        job_skills.append(extracted_skills)
       
        job_driver.quit()
        
    driver.quit()

    job_data = pd.DataFrame({'Date': post_date, 'Company': company_list,'Post': post_list, 'Location': job_location,'Link' : link_list})
    job_data.to_csv('LinkedIn_jobs.csv', index=0)
    
    return job_skills


In [174]:
if __name__ == "__main__":
    keyword = "data%20engineer"
    num_pages = 1
    
    job_skills = scrape_linkedin_jobs(keyword, num_pages)
    
    print(f'Data engineer jobs: {len(job_skills)}')

    flattened_skills = [skill for sublist in job_skills for skill in sublist]
    skill_counts = Counter(flattened_skills)
    top_skills = skill_counts.most_common(30)

    for skill, count in top_skills:
        print(f'{skill}: {count}')

Scrolling Loading Function
---------------------------
Scrolling to load more jobs - Page 1
After parsing url within Scrape function
---------------------------
location:Los Angeles, CA
Time:4 days ago
Junior Data Engineer at Los Angeles Dodgers
Job Link: https://www.linkedin.com/jobs/view/junior-data-engineer-at-los-angeles-dodgers-3728707694?refId=i13GNx%2B1C%2BKwJKqu9VY%2BaA%3D%3D&trackingId=Saw1rYnAdSj7uiGIbRwXeA%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card
['PostgreSQL']
---------------------------
---------------------------
location:Detroit, MI
Time:2 weeks ago
Data Engineer at DTE Energy
Job Link: https://www.linkedin.com/jobs/view/data-engineer-at-dte-energy-3719944894?refId=i13GNx%2B1C%2BKwJKqu9VY%2BaA%3D%3D&trackingId=ffIdf7PrNa6H2PwZS%2BseGA%3D%3D&position=2&pageNum=0&trk=public_jobs_jserp-result_search-card
['Databricks']
---------------------------
---------------------------
location:Highlands Ranch, CO
Time:1 month ago
Data Engineer Intern at Lock